In [ ]:
import pandas as pd
import statsmodels.api as sm

# Part 1: Portfolio construction based on inflation betas

# Load data with stock prices and inflation (must have 'Date' and 'Inflation' columns)
df = pd.read_excel("monthly_prices_and_inflation.xlsx", parse_dates=['Date']).set_index('Date')

# Extract the inflation series and calculate stock returns
infl = df.pop('Inflation')                      # Remove and store the inflation column
rets = df.pct_change().dropna()                 # Calculate monthly returns from price data

# Align inflation with returns and add a constant for regression
X = sm.add_constant(infl.loc[rets.index])

# Run regression of each stock's return on inflation to get inflation betas
betas = {
    stock: sm.OLS(rets[stock], X, missing='drop').fit().params['Inflation']
    for stock in rets.columns
}
beta_s = pd.Series(betas)                       # Convert beta dictionary to Series for sorting

# Sort stocks by beta and divide into 20 equal-sized portfolios
sorted_idx = beta_s.sort_values(ascending=False).index
group_size = len(sorted_idx) // 20
portfolios = [sorted_idx[i*group_size:(i+1)*group_size] for i in range(20)]

# Calculate equal-weighted monthly returns for each portfolio
port_rets = pd.concat(
    {f'P{p+1:02d}': rets[grp].mean(axis=1) for p, grp in enumerate(portfolios)},
    axis=1
)

# Combine inflation and portfolio returns for saving
combined = pd.concat([infl.loc[port_rets.index].rename('Inflation'), port_rets], axis=1)
beta_s.to_csv("inflation_betas.csv")            # Save inflation betas
combined.to_excel("monthly_portfolio_returns.xlsx", index=True)  # Save returns
print("✔ Saved: 'inflation_betas.csv' and 'monthly_portfolio_returns.xlsx'")

# Part 2: Run regressions of portfolio returns on inflation

# Load the saved file containing inflation and portfolio returns
df2 = pd.read_excel("monthly_portfolio_returns.xlsx")
inflation = df2.iloc[:, 1]                      # Inflation is the second column
portfolio_returns = df2.iloc[:, 2:]             # Portfolios start from third column
X = sm.add_constant(inflation)

# Regress each portfolio return on inflation and print results
for name in portfolio_returns.columns:
    y = portfolio_returns[name]
    model = sm.OLS(y, X).fit()
    print(f"\nRegression results for {name}:")
    print(model.summary())

# Part 3: Estaimate Fama and French factor model for portfolios

# Load the Excel file
data = pd.read_excel("data.xlsx")

# Define the dependent variables: portfolio excess returns
# These should be 20 columns (e.g. R1, R2, ..., R20) representing excess returns
portfolio_columns = ['P1', 'P2', 'P3', 'P4', 'P5',
                     'P6', 'P7', 'P8', 'P9', 'P10',
                     'P11', 'P12', 'P13', 'P14', 'P15',
                     'P16', 'P17', 'P18', 'P19', 'P20']

# Define the Fama-French 5 factors: MKT-RF, SMB, HML, RMW, CMA
factor_columns = ['MKT_RF', 'SMB', 'HML', 'RMW', 'CMA']

# Run OLS regression for each portfolio return ~ 5 factors
for portfolio in portfolio_columns:
    y = data[portfolio]  # Dependent variable: portfolio excess return
    X = sm.add_constant(data[factor_columns])  # Independent variables: 5 factors + intercept
    model = sm.OLS(y, X).fit()
    
    print(f"Regression results for portfolio: {portfolio}")
    print(model.summary())
    print("\n" + "="*80 + "\n")
